## Product Rationalization LP Problem

In [1]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import pandas as pd
import numpy as np

The Molokai Nut Company (MNC) makes four different products from macadamia nuts grown in the Hawaiian Islands:  chocolate-coated whole nuts (Whole), chocolate-coated nut clusters (Cluster), chocolate-coated nut crunch bars (Crunch) and plain roasted nuts (Roasted).

To meet marketing demands for the coming week, MNC needs to produce at least 1,000 pounds of the Whole product, between 400 and 500 pounds of the Cluster product, no more than 150 pounds of the Crunch product and no more than 200 pounds of the Roasted product.

Each pound of the Whole, Cluster, Crunch and Roasted product contains, respectively, 60%, 40%, 20% and 100% macadamia nuts with the remaining weight made up of chocolate coating.  The company has 1100 pounds of nuts and 800 pounds of chocolate available for use in the next week.  

The various products are made using four different machines that hull the nuts, roast the nuts, coat the nuts in chocolate, and package the products.  Table 1 summarizes the time required in minutes for each product on each machine.  Each machine has 60 hours of production time available in the coming week.

**Table 1: Machine Minutes Required per Pound**

|Machine  |Whole    |Cluster     |Crunch     |Roasted      |
|:-------:|:-------:|:----------:|:---------:|:-----------:|
|Hulling  |1.00     |1.00        |1.00       |1.00         |
|Roasting |2.00     |1.50        |1.00       |4.00         |
|Coating  |1.00     |0.70        |0.20       |0            |
|Packaging|2.50     |1.60        |1.25       |1.00         |

The company's controller recently presented management with a financial summary of MNC's average weekly operations over the past quarter.  The Variable Margin per pound for each of the products is shown below.

|Product      |VM/lb       |
|:-----------:|:----------:|
|Whole        |\\$1.93     |
|Cluster      |\\$1.04     |
|Crunch       |\\$1.15     |
|Roasted      |\\$1.33     |

A. Build a linear programming model to identify the production levels for MNC's four products that maximize total VM.

B. If MNC wanted to decrease the production of any product, which product would you recommend and why?

C. Which machine capacities would you recommend the company look into expanding?  If they can only expand one machine capacity, which machine should they target?

In [2]:
data = pd.DataFrame({'product': ['whole', 'cluster', 'crunch', 'roasted'],
                     'vm': [1.93, 1.04, 1.15, 1.33],
                     'hulling': [1, 1, 1, 1],
                     'roasting': [2, 1.5, 1, 4],
                     'coating': [1, .7, .2, 0],
                     'packaging': [2.5, 1.6, 1.25, 1],
                     'nuts': [.6, .4, .2, 1],
                     'chocolate': [.4, .6, .8, 0],
                    'max_needed': [100000, 500, 150, 200],
                    'min_needed': [1000, 400, 0, 0]})
data = data.set_index('product')
data

,vm,hulling,roasting,coating,packaging,nuts,chocolate,max_needed,min_needed
product,,,,,,,,,
whole,1.93,1,2.0,1.0,2.50,0.6,0.4,100000,1000
cluster,1.04,1,1.5,0.7,1.60,0.4,0.6,500,400
crunch,1.15,1,1.0,0.2,1.25,0.2,0.8,150,0
roasted,1.33,1,4.0,0.0,1.00,1.0,0.0,200,0


In [3]:
machines = data.columns[1:5]
ingredients = data.columns[5:7]

capacities = dict(zip(machines, [3600, 3600, 3600, 3600]))
materials = dict(zip(ingredients, [1100, 800]))

materials

{'nuts': 1100, 'chocolate': 800}

In [4]:
model = LpProblem("Product_Rationalization", LpMaximize)

prod = LpVariable.dicts("qty", data.index)
prod

{'whole': qty_whole,
 'cluster': qty_cluster,
 'crunch': qty_crunch,
 'roasted': qty_roasted}

In [5]:
model += lpSum(data.loc[p, 'vm'] * prod[p] for p in data.index )
model

Product_Rationalization:
MAXIMIZE
1.04*qty_cluster + 1.15*qty_crunch + 1.33*qty_roasted + 1.93*qty_whole + 0.0
VARIABLES
qty_cluster free Continuous
qty_crunch free Continuous
qty_roasted free Continuous
qty_whole free Continuous

In [6]:
for m in machines :
        model +=  (lpSum(data.loc[p, m] * prod[p] for p in data.index) <= capacities[m], 'capacity_' + str(m)) 

        
for i in ingredients :
        model +=  (lpSum(data.loc[p, i] * prod[p] for p in data.index) <= materials[i], 'materials_' + str(i))         

        
for p in data.index:
    model += (prod[p] >= data.loc[p, 'min_needed'], 'lower_' + str(p))
    model += (prod[p] <= data.loc[p, 'max_needed'], 'upper_' + str(p))
        
model            

Product_Rationalization:
MAXIMIZE
1.04*qty_cluster + 1.15*qty_crunch + 1.33*qty_roasted + 1.93*qty_whole + 0.0
SUBJECT TO
capacity_hulling: qty_cluster + qty_crunch + qty_roasted + qty_whole <= 3600

capacity_roasting: 1.5 qty_cluster + qty_crunch + 4 qty_roasted + 2 qty_whole
 <= 3600

capacity_coating: 0.7 qty_cluster + 0.2 qty_crunch + qty_whole <= 3600

capacity_packaging: 1.6 qty_cluster + 1.25 qty_crunch + qty_roasted
 + 2.5 qty_whole <= 3600

materials_nuts: 0.4 qty_cluster + 0.2 qty_crunch + qty_roasted + 0.6 qty_whole
 <= 1100

materials_chocolate: 0.6 qty_cluster + 0.8 qty_crunch + 0.4 qty_whole <= 800

lower_whole: qty_whole >= 1000

upper_whole: qty_whole <= 100000

lower_cluster: qty_cluster >= 400

upper_cluster: qty_cluster <= 500

lower_crunch: qty_crunch >= 0

upper_crunch: qty_crunch <= 150

lower_roasted: qty_roasted >= 0

upper_roasted: qty_roasted <= 200

VARIABLES
qty_cluster free Continuous
qty_crunch free Continuous
qty_roasted free Continuous
qty_whole free Con

In [7]:
model.solve()

1

In [8]:
model.objective.value()

2839.075

#### A. Build a linear programming model to identify the production levels for MNC's four products that maximize total VM.

In [9]:
for v in model.variables(): print(f"{v.name}: {v.varValue}")

qty_cluster: 400.0
qty_crunch: 150.0
qty_roasted: 197.5
qty_whole: 1030.0


#### B. If MNC wanted to decrease the production of any product, which product would you recommend and why?

Cluster lower bound has negative shadow price => increase profit by making fewer pounds of Cluster

#### C. Which machine capacities would you recommend the company look into expanding? If they can only expand one machine capacity, which machine should they target?

Packaging and roasting.  First priority packaging.  Each additional unit of packaging time adds \\$0.6325 to VM.

In [20]:
for c in model.constraints.values(): print(f"{c.name}: {c.pi}")

capacity_hulling: -0.0
capacity_roasting: 0.174375
capacity_coating: -0.0
capacity_packaging: 0.6325
materials_nuts: -0.0
materials_chocolate: -0.0
lower_whole: -0.0
upper_whole: -0.0
lower_cluster: -0.2335625
upper_cluster: -0.0
lower_crunch: -0.0
upper_crunch: 0.185
lower_roasted: -0.0
upper_roasted: -0.0
